In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os

# -------------------------
# Step 1. File selection
# -------------------------
root = Tk()
root.withdraw()  # hide root window

file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)
if not file_path:
    raise SystemExit("❌ No file selected.")
print(f"Selected file: {file_path}")


# -------------------------
# Step 2. Custom dialog for Year & Month
# -------------------------
def ask_year_month():
    def on_ok():
        nonlocal selected_year, selected_month
        selected_year = year_var.get()
        selected_month = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    # Year dropdown
    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))  # default
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    # Month dropdown
    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])  # default
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)

    win.wait_window()  # wait for user to close
    return int(selected_year), selected_month


selected_year, selected_month = ask_year_month()
print(f"Selected: {selected_month} {selected_year}")


# -------------------------
# Step 3. Processing
# -------------------------
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")
df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()

# Current selection
sel_year = selected_year
sel_month = list(calendar.month_name).index(selected_month)
curr_date = datetime(sel_year, sel_month, 1)

# Horizon (24 months from Jan of selected year)
start_date = datetime(sel_year, 1, 1)
horizon = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in horizon]

output_records = []

def get_key(row):
    if pd.notna(row["PID"]):
        return ("PID", row["PID"], row["GCB"], row["Business Framework"])
    else:
        return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month")

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    for label in month_labels:
        record[label] = None

    last_fte = None
    for month in month_labels:
        month_dt = datetime.strptime(month, "%b-%y")
        row = group[group["Period Month"] == month_dt]
        if not row.empty:
            last_fte = row["FTE"].iloc[0]
        if last_fte is not None:
            record[month] = last_fte
        if month_dt <= curr_date and record[month] is not None:
            record["FTE"] = record[month]

    output_records.append(record)

out_df = pd.DataFrame(output_records)

# Sort output
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

# Save
out_file = os.path.join(
    os.path.dirname(file_path),
    f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")
